In [1]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
np.set_printoptions(precision=4, suppress=True)

# 11.6 리샘플링과 빈도 변환

In [2]:
# 리샘플링은 시계열의 빈도를 변환하는 과정을 일컫는다. 
# 상위 빈도의 데이터를 하위 빈도로 집계하는 것을 다운샘플링이라고 하며 반대 과정을 업샘플링이라고 한다.

In [4]:
# 모든 리샘플링이 이 두 가지 범주에 들어가지는 않는다. 
# 예를 들어 W-WED(수요일을 기준으로 한 주간)를 W-FRI로 변경하는 것은 업샘플링도 다운샘플링도 아니다.

In [5]:
# pandas 객체는 resample 메서드를 가지고 있는데, 빈도 변환과 관련된 모든 작업에서 유용하게 사용되는 메서드다. 
# resample은 groupby와 비슷한 API를 가지고 있는데 resample을 호출해서 데이터를 그룹 짓고 요약함수를 적용하는 식이다.

In [6]:
rng = pd.date_range("2000-01-01", periods=100, freq="D")

In [7]:
ts = pd.Series(np.random.randn(len(rng)), index=rng)

In [8]:
ts

2000-01-01   -0.204708
2000-01-02    0.478943
2000-01-03   -0.519439
2000-01-04   -0.555730
2000-01-05    1.965781
                ...   
2000-04-05    0.795253
2000-04-06    0.118110
2000-04-07   -0.748532
2000-04-08    0.584970
2000-04-09    0.152677
Freq: D, Length: 100, dtype: float64

In [9]:
ts.resample("M").mean()

2000-01-31    0.304166
2000-02-29   -0.404772
2000-03-31    0.021947
2000-04-30    0.554480
Freq: M, dtype: float64

In [10]:
ts.resample("M", kind="period").mean()

2000-01    0.304166
2000-02   -0.404772
2000-03    0.021947
2000-04    0.554480
Freq: M, dtype: float64

In [12]:
# resample은 유연한 고수준의 메서드로, 매우 큰 시계열 데이터를 처리할 수 있다. 
# 다음 절부터 몇 가지 예제를 통해 자세한 내용을 살펴보자. 
# [표 11-5]에 resample 메서드의 인자 일부를 설명했다. 페이지 464

# 11.6.1 다운샘플링

In [13]:
# 시계열 데이터를 규칙적인 하위 빈도로 집계하는 일은 특별한 일이 아니다. 
# 집계할 데이터는 고정 빈도를 가질 필요가 없으며 잘라낸 시계열 조각의 크기를 원하는 빈도로 정의한다.
# 예를 들어 "M"이나 "BM" 같은 월간 빈도로 변환하려면 데이터를 월 간격으로 나눠야 한다.

In [ ]:
# 각 간격은 한쪽이 열려 있게 되는데, 이 말은 하나의 간격에서 양끝 중 한쪽만 포함된다는 뜻이다. 
# 그러면 각 간격의 모음이 전체 시계열이 된다. 

- resample을 사용해서 데이터를 다운샘플링할 때 고려해야 할 사항이 몇 가지 있다.
    - 각 간격의 양끝 중에서 어느 쪽을 닫아둘 것인가
    - 집계하려는 구간의 라벨을 간격의 시작으로 할지 끝으로 할지 여부

In [14]:
# 분 단위 데이터를 통해 좀 더 알아보자.

In [15]:
rng = pd.date_range("2000-01-01", periods=12, freq="T")

In [16]:
ts = pd.Series(np.arange(12), index=rng)

In [17]:
ts

2000-01-01 00:00:00     0
2000-01-01 00:01:00     1
2000-01-01 00:02:00     2
2000-01-01 00:03:00     3
2000-01-01 00:04:00     4
2000-01-01 00:05:00     5
2000-01-01 00:06:00     6
2000-01-01 00:07:00     7
2000-01-01 00:08:00     8
2000-01-01 00:09:00     9
2000-01-01 00:10:00    10
2000-01-01 00:11:00    11
Freq: T, dtype: int32

In [18]:
# 이 데이터를 5분 단위로 묶어서 각 그룹의 합을 집계해보자.

In [19]:
ts.resample("5min", closed="right").sum()

1999-12-31 23:55:00     0
2000-01-01 00:00:00    15
2000-01-01 00:05:00    40
2000-01-01 00:10:00    11
Freq: 5T, dtype: int32

In [20]:
# 인자로 넘긴 빈도는 5분 단위로 증가하는 그룹의 경계를 정의한다.
# 기본적으로 시작값을 그룹의 왼쪽에 포함시키므로 00:00의 값은 첫 번째 그룹의 00:00부터 00:05까지의 값을 집계한다.
# closed="right"를 넘기면 시작값을 그룹의 오른쪽에 포함시킨다.

In [21]:
ts.resample("5min", closed="right").sum()

1999-12-31 23:55:00     0
2000-01-01 00:00:00    15
2000-01-01 00:05:00    40
2000-01-01 00:10:00    11
Freq: 5T, dtype: int32

In [22]:
# 결과로 반환된 시계열은 각 그룹의 왼쪽 타임스탬프가 라벨로 지정되었다.
# label="right"를 넘겨서 각 그룹의 오른쪽 값을 라벨로 사용할 수 있다.

In [23]:
ts.resample("5min", closed="right", label="right").sum()

2000-01-01 00:00:00     0
2000-01-01 00:05:00    15
2000-01-01 00:10:00    40
2000-01-01 00:15:00    11
Freq: 5T, dtype: int32

In [24]:
# [그림 11-3]에 5분 단위 리샘플링에서의 closed와 label을 나타냈다. 페이지 466

In [25]:
# 반환된 결과의 색인을 특정 크기만큼 이동시키고 싶은 경우, 
# 즉 그룹의 오른쪽 끝에서 1초를 빼서 타임스탬프가 참조하는 간격을 좀 더 명확히 보여주고 싶은 경우에는 loffset 메서드에 문자열이나 날짜 오프셋을 넘기면 된다.

In [27]:
ts.resample("5min", closed="right",
            label="right", loffset="-1s").sum()

C:\Users\ahnjh\AppData\Local\Temp\ipykernel_15444\4215765596.py:1: FutureWarning: 'loffset' in .resample() and in Grouper() is deprecated.

>>> df.resample(freq="3s", loffset="8H")

becomes:

>>> from pandas.tseries.frequencies import to_offset
>>> df = df.resample(freq="3s").mean()
>>> df.index = df.index.to_timestamp() + to_offset("8H")

  ts.resample("5min", closed="right",


1999-12-31 23:59:59     0
2000-01-01 00:04:59    15
2000-01-01 00:09:59    40
2000-01-01 00:14:59    11
Freq: 5T, dtype: int32

In [28]:
# loffset 대신 반환된 결과에 shift 메서드를 사용해도 같은 결과를 얻을 수 있다.

- OHLC 리샘플링

In [29]:
# 금융 분야에서 시계열 데이터를 집계하는 아주 흔한 방식은 각 버킷에 대해 4가지 값을 계산하는 것이다.
# 이 4가지 값은 시가, 고가, 저가, 종가이며, 이를 OHLC라고 한다. 
# how="ohlc"를 넘겨서 한 번에 이 값을 담고 있는 컬럼을 가지는 DataFrame을 얻을 수 있다.

In [30]:
ts.resample("5min").ohlc()

,open,high,low,close
2000-01-01 00:00:00,0,4,0,4
2000-01-01 00:05:00,5,9,5,9
2000-01-01 00:10:00,10,11,10,11


# 11.6.2 업샘플링과 보간

In [31]:
# 하위 빈도에서 상위 빈도로 변환할 때는 집계가 필요하지 않다.
# 주간 데이터를 담고 있는 DataFrame을 살펴보자.

In [32]:
frame = pd.DataFrame(np.random.randn(2, 4),
                     index=pd.date_range("1/1/2000", periods=2,
                                         freq="W-WED"),
                     columns=["Colorado", "Texas", "New York", "Ohio"])

In [33]:
frame

,Colorado,Texas,New York,Ohio
2000-01-05,-1.565657,-0.562540,-0.032664,-0.929006
2000-01-12,-0.482573,-0.036264,1.095390,0.980928


In [34]:
# 이 데이터에 요약함수를 사용하면 그룹당 하나의 값이 들어가고 그 사이에 결측치가 들어간다.
# asfreq 메서드를 이용해서 어떤 요약함수도 사용하지 않고 상위 빈도로 리샘플링해보자.

In [35]:
df_daily = frame.resample("D").asfreq()

In [36]:
df_daily

,Colorado,Texas,New York,Ohio
2000-01-05,-1.565657,-0.562540,-0.032664,-0.929006
2000-01-06,NaN,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN,NaN
2000-01-08,NaN,NaN,NaN,NaN
2000-01-09,NaN,NaN,NaN,NaN
2000-01-10,NaN,NaN,NaN,NaN
2000-01-11,NaN,NaN,NaN,NaN
2000-01-12,-0.482573,-0.036264,1.095390,0.980928


In [37]:
# 수요일이 아닌 요일에는 이전 값을 채워서 보간을 수행한다고 가정하자.
# fillna와 reindex 메서드에서 사용했던 보간 메서드를 리샘플링에서도 사용할 수 있다.

In [38]:
frame.resample("D").ffill()

,Colorado,Texas,New York,Ohio
2000-01-05,-1.565657,-0.562540,-0.032664,-0.929006
2000-01-06,-1.565657,-0.562540,-0.032664,-0.929006
2000-01-07,-1.565657,-0.562540,-0.032664,-0.929006
2000-01-08,-1.565657,-0.562540,-0.032664,-0.929006
2000-01-09,-1.565657,-0.562540,-0.032664,-0.929006
2000-01-10,-1.565657,-0.562540,-0.032664,-0.929006
2000-01-11,-1.565657,-0.562540,-0.032664,-0.929006
2000-01-12,-0.482573,-0.036264,1.095390,0.980928


In [39]:
# limit 옵션을 사용해서 보간법을 적용할 범위를 지정하는 것도 가능하다.

In [40]:
frame.resample("D").ffill(limit=2)

,Colorado,Texas,New York,Ohio
2000-01-05,-1.565657,-0.562540,-0.032664,-0.929006
2000-01-06,-1.565657,-0.562540,-0.032664,-0.929006
2000-01-07,-1.565657,-0.562540,-0.032664,-0.929006
2000-01-08,NaN,NaN,NaN,NaN
2000-01-09,NaN,NaN,NaN,NaN
2000-01-10,NaN,NaN,NaN,NaN
2000-01-11,NaN,NaN,NaN,NaN
2000-01-12,-0.482573,-0.036264,1.095390,0.980928


In [41]:
# 특히 새로운 날짜 색인은 이전 색인과 겹쳐질 필요가 전혀 없다.

In [42]:
frame.resample("W-THU").ffill()

,Colorado,Texas,New York,Ohio
2000-01-06,-1.565657,-0.562540,-0.032664,-0.929006
2000-01-13,-0.482573,-0.036264,1.095390,0.980928


# 11.6.3 기간 리샘플링

In [43]:
# 기간으로 색인된 데이터를 리샘플링하는 것은 타임스탬프와 유사하다.

In [44]:
frame = pd.DataFrame(np.random.randn(24, 4),
                     index=pd.period_range("1-2000", "12-2001",
                                         freq="M"),
                     columns=["Colorado", "Texas", "New York", "Ohio"])

In [45]:
frame[:5]

,Colorado,Texas,New York,Ohio
2000-01,-0.589488,1.581700,-0.528735,0.457002
2000-02,0.929969,-1.569271,-1.022487,-0.402827
2000-03,0.220487,-0.193401,0.669158,-1.648985
2000-04,-2.252797,-1.166832,0.353607,0.702110
2000-05,-0.274569,-0.139142,0.107657,-0.606545


In [47]:
annual_frame = frame.resample("A-DEC").mean()

In [48]:
annual_frame

,Colorado,Texas,New York,Ohio
2000,0.024752,-0.089078,-0.131906,-0.313531
2001,0.113353,0.025106,0.033542,0.193194


In [49]:
# 업샘플링은 asfreq 메서드처럼 리샘플링하기 전에 새로운 빈도에서 구간의 끝을 어느 쪽에 두어야 할 지 미리 결정해야 한다.
# convention 인자의 기본값은 "start"이지만 "end"로 지정할 수도 있다.

In [50]:
# Q-DEC: 12월을 연도마감으로 하는 분기 주기

In [55]:
annual_frame.resample("Q-DEC").ffill()

,Colorado,Texas,New York,Ohio
2000Q1,0.024752,-0.089078,-0.131906,-0.313531
2000Q2,0.024752,-0.089078,-0.131906,-0.313531
2000Q3,0.024752,-0.089078,-0.131906,-0.313531
2000Q4,0.024752,-0.089078,-0.131906,-0.313531
2001Q1,0.113353,0.025106,0.033542,0.193194
2001Q2,0.113353,0.025106,0.033542,0.193194
2001Q3,0.113353,0.025106,0.033542,0.193194
2001Q4,0.113353,0.025106,0.033542,0.193194


In [56]:
annual_frame.resample("Q-DEC", convention="end").ffill()

,Colorado,Texas,New York,Ohio
2000Q4,0.024752,-0.089078,-0.131906,-0.313531
2001Q1,0.024752,-0.089078,-0.131906,-0.313531
2001Q2,0.024752,-0.089078,-0.131906,-0.313531
2001Q3,0.024752,-0.089078,-0.131906,-0.313531
2001Q4,0.113353,0.025106,0.033542,0.193194


- 기간의 업샘플링과 다운샘플링은 좀 더 엄격하다.
    - 다운샘플링의 경우 대상 빈도는 반드시 원본 빈도의 하위 기간이어야 한다.
    - 업샘플링의 경우 대상 빈도는 반드시 원본 빈도의 상위 기간이어야 한다.

In [57]:
# 위 조건을 만족하지 않으면 예외가 발생한다. 이 예외는 주로 분기, 연간, 주간 빈도에서 발생한다.
# 예를 들어 Q-MAR로 정의된 기간은 A-MAR, A-JUN, A-SEP, A-DEC로만 이루어져 있다.

In [58]:
annual_frame.resample("Q-MAR").ffill()

,Colorado,Texas,New York,Ohio
2000Q4,0.024752,-0.089078,-0.131906,-0.313531
2001Q1,0.024752,-0.089078,-0.131906,-0.313531
2001Q2,0.024752,-0.089078,-0.131906,-0.313531
2001Q3,0.024752,-0.089078,-0.131906,-0.313531
2001Q4,0.113353,0.025106,0.033542,0.193194
2002Q1,0.113353,0.025106,0.033542,0.193194
2002Q2,0.113353,0.025106,0.033542,0.193194
2002Q3,0.113353,0.025106,0.033542,0.193194
